In [69]:
import pandas as pd
import utils
import plotly.graph_objects as go
import instrument

In [70]:
import yfinance as yf #  pip install yfinance 
symbol='EURUSD=X'
start_date='2024-3-1'
end_date='2024-4-20'
interval='1h'
data= yf.download(tickers=symbol,start=start_date,end=end_date,interval=interval)
data=pd.DataFrame(data)
data


[*********************100%%**********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Datetime,,,,,,
2024-03-01 00:00:00+00:00,1.080964,1.081549,1.080847,1.081432,1.081432,0
2024-03-01 01:00:00+00:00,1.081432,1.082368,1.081432,1.082368,1.082368,0
2024-03-01 02:00:00+00:00,1.082368,1.082485,1.082017,1.082251,1.082251,0
2024-03-01 03:00:00+00:00,1.082134,1.082368,1.082017,1.082017,1.082017,0
2024-03-01 04:00:00+00:00,1.082017,1.082017,1.081549,1.081666,1.081666,0
...,...,...,...,...,...,...
2024-04-19 18:00:00+01:00,1.065644,1.066098,1.065417,1.065871,1.065871,0
2024-04-19 19:00:00+01:00,1.065757,1.065757,1.064963,1.065417,1.065417,0
2024-04-19 20:00:00+01:00,1.065303,1.065757,1.065303,1.065757,1.065757,0


In [71]:
# Reset the index to move the datetime information into a column
data.reset_index(inplace=True)
data.rename(columns={    'Datetime': 'time','Open': 'open', 'High': 'high', 'Low': 'low', 'Close': 'close', 'Volume': 'volume','Adj Close':'adj close'}, inplace=True)
data['time']=data['time'].astype('str')
data['time']=data['time'].str.replace(":00:00+00:00", "")
data.head()


,time,open,high,low,close,adj close,volume
0,2024-03-01 00,1.080964,1.081549,1.080847,1.081432,1.081432,0
1,2024-03-01 01,1.081432,1.082368,1.081432,1.082368,1.082368,0
2,2024-03-01 02,1.082368,1.082485,1.082017,1.082251,1.082251,0
3,2024-03-01 03,1.082134,1.082368,1.082017,1.082017,1.082017,0
4,2024-03-01 04,1.082017,1.082017,1.081549,1.081666,1.081666,0


In [72]:
pair = "EUR_USD"
granularity = "H1"
ma_list = [16,64]
i_pair = instrument.Instrument.get_instrument_by_name("EUR_USD")

In [73]:
#df = pd.read_pickle(utils.get_his_data_filename(pair, granularity))
non_cols = ['time', 'volume']
mod_cols = [x for x in data.columns if x not in non_cols]
data[mod_cols] = data[mod_cols].apply(pd.to_numeric)

In [81]:
data.rename(columns={    'time': 'time','open': 'mid_o', 'high': 'mid_h', 'low': 'mid_l', 'close':'mid_c', 'Volume': 'volume','Adj Close':'adj close'}, inplace=True)

data.head()

,time,mid_o,mid_h,mid_l,mid_c,adj close,volume
0,2024-03-01 00,1.080964,1.081549,1.080847,1.081432,1.081432,0
1,2024-03-01 01,1.081432,1.082368,1.081432,1.082368,1.082368,0
2,2024-03-01 02,1.082368,1.082485,1.082017,1.082251,1.082251,0
3,2024-03-01 03,1.082134,1.082368,1.082017,1.082017,1.082017,0
4,2024-03-01 04,1.082017,1.082017,1.081549,1.081666,1.081666,0


In [82]:
df_ma = data[['time', 'mid_o', 'mid_h', 'mid_l', 'mid_c']].copy()
for ma in ma_list:  
    df_ma[f'MA_{ma}'] = df_ma.mid_c.rolling(window=ma).mean()
df_ma.dropna(inplace=True)
df_ma.reset_index(drop=True, inplace=True)

In [83]:
df_ma.head()

,time,mid_o,mid_h,mid_l,mid_c,MA_16,MA_64
0,2024-03-05 16,1.087429,1.087429,1.086130,1.086484,1.085431,1.084386
1,2024-03-05 17,1.086484,1.086720,1.086130,1.086248,1.085489,1.084462
2,2024-03-05 18,1.086248,1.086248,1.085541,1.085541,1.085504,1.084511
3,2024-03-05 19,1.085541,1.085894,1.085423,1.085541,1.085489,1.084563
4,2024-03-05 20,1.085658,1.086130,1.085187,1.086130,1.085534,1.084627


In [84]:
def is_trade(row):
    if row.DIFF >= 0 and row.DIFF_PREV < 0:
        return True
    if row.DIFF <= 0 and row.DIFF_PREV > 0:
        return True
    return False

In [85]:
df_ma['DIFF'] = df_ma.MA_16 - df_ma.MA_64
df_ma['DIFF_PREV'] = df_ma.DIFF.shift(1)
df_ma['IS_TRADE'] = df_ma.apply(is_trade, axis=1)
df_trades = df_ma[df_ma.IS_TRADE==True].copy()

In [86]:
df_trades.head()

,time,mid_o,mid_h,mid_l,mid_c,MA_16,MA_64,DIFF,DIFF_PREV,IS_TRADE
100,2024-03-11 21,1.092777,1.093016,1.092657,1.093016,1.093449,1.093562,-1.126211e-04,0.000014,True
142,2024-03-13 15,1.094691,1.095050,1.094212,1.094571,1.093412,1.093377,3.533252e-05,-0.000056,True
166,2024-03-14 15,1.090037,1.090275,1.088613,1.089087,1.093579,1.093579,-6.072223e-07,0.000343,True
268,2024-03-20 22,1.092538,1.093494,1.092538,1.093494,1.087625,1.087361,2.635811e-04,-0.000054,True
293,2024-03-21 23,1.086484,1.086602,1.086366,1.086484,1.088129,1.088145,-1.559034e-05,0.000413,True


In [87]:
df_plot = df_ma.iloc[0:24].copy()

In [88]:
df_plot.shape

(24, 10)

In [89]:
fig = go.Figure()
fig.add_trace(go.Candlestick(
    x=df_plot.time, open=df_plot.mid_o, high=df_plot.mid_h, low=df_plot.mid_l, close=df_plot.mid_c,
    line=dict(width=1), opacity=1,
    increasing_fillcolor='#24A06B',
    decreasing_fillcolor="#CC2E3C",
    increasing_line_color='#2EC886',  
    decreasing_line_color='#FF3A4C'
))
for ma in ma_list:  
    col = f"MA_{ma}"
    fig.add_trace(go.Scatter(x=df_plot.time, 
        y=df_plot[col],
        line=dict(width=2),
        line_shape='spline',
        name=col
        ))
fig.update_layout(width=1000,height=400,
    margin=dict(l=10,r=10,b=10,t=10),
    font=dict(size=10,color="#e1e1e1"),
    paper_bgcolor="#1e1e1e",
    plot_bgcolor="#1e1e1e")
fig.update_xaxes(
    gridcolor="#1f292f",
    showgrid=True,fixedrange=True,rangeslider=dict(visible=False)
)
fig.update_yaxes(
    gridcolor="#1f292f",
    showgrid=True
)
fig.show()